* a.	Извлекать реплики с приветствием – где менеджер поздоровался. 
* b.	Извлекать реплики, где менеджер представил себя. 
* c.	Извлекать имя менеджера. 
* d.	Извлекать название компании. 
* e.	Извлекать реплики, где менеджер попрощался.
* f.	Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


Лучшие модели для задачи NER
* surdan/LaBSE_ner_nerel
* xlm-roberta-large-finetuned-conll03-english

In [1]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import re
import numpy as np

In [2]:
data = pd.read_csv('/kaggle/input/bewiseai/test_data.csv')

Создаем новые столбцы

In [3]:
data['greeting'] = False
data['farewell'] = False
data['naming'] = False
data['person'] = ''
data['company'] = ''

In [4]:
data

,dlg_id,line_n,role,text,greeting,farewell,naming,person,company
0,0,0,client,Алло,False,False,False,,
1,0,1,manager,Алло здравствуйте,False,False,False,,
2,0,2,client,Добрый день,False,False,False,,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,False,,
4,0,4,client,Ага,False,False,False,,
...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,False,,
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,False,,
477,5,140,client,Спасибо спасибо,False,False,False,,
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,False,,


In [5]:
def classify_sentence(row, sentence_type: str):
    """с помощью регулярных выражений определить является ли предложение приветствием, прощанием или представлением"""
    reg_types={
        'naming': r"зовут|имя|звать",
        'greeting': r"привет|добр|здравствуй",
        'farewell': r"свидан|пока|встреч|прощ|хорошего|счастлив|доброго"
    }
    text = row.text.lower()
    match = re.search(reg_types[sentence_type], text)
    if match:
        data.at[row.name, sentence_type] = True
#         print(text)

Проверка нескольких первых предложений на принадлжность к приветствию, предствлению. И нескольких последних на прощание. И заполнениие соответствующих столбцов

In [6]:
NUM_OF_BEGIN_SENTENCES = 4
NUM_OF_END_SENTENCES = 4
for dlg_id in data.dlg_id.unique():
    dlg = data[data.dlg_id == dlg_id]
    dlg = dlg.sort_values('line_n', ascending=True)
    dlg = dlg[dlg.role == 'manager']
    for idx in range(NUM_OF_BEGIN_SENTENCES):
        row = dlg.iloc[idx] 
        classify_sentence(row, 'greeting')
        classify_sentence(row, 'naming')
    for idx in range(dlg.shape[0] - NUM_OF_END_SENTENCES, dlg.shape[0]):
        row = dlg.iloc[idx]
        classify_sentence(row, 'farewell')
        

Создание токенизатора и классификатора, для определения сущности слов (человек, организация)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)


Downloading:   0%|          | 0.00/852 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

In [8]:
def get_person_company(row, classifier) -> tuple: 
    """ определение сущности слов (человек, организация) и получение имени менеджера и названия компании """
    text = row.text.lower()
    output = classifier(text)
    person = ''
    company = ''
    for part in output:
        if (part['entity'] == 'I-PER'):
            person += part['word']
        if (part['entity'] == 'B-ORG' or part['entity'] == 'I-ORG'):
            company += part['word']
    return person, company

 Получение имени менеджера и названия компании

In [9]:
namings = data[data.naming == True]

for idx in range(namings.shape[0]):
    row = namings.iloc[idx]
    person, company = get_person_company(row, classifier)
    data.at[row.name, 'person'] = person
    data.at[row.name, 'company'] = company

In [10]:
data[data.greeting==True]

,dlg_id,line_n,role,text,greeting,farewell,naming,person,company
1,0,1,manager,Алло здравствуйте,True,False,False,,
110,1,1,manager,Алло здравствуйте,True,False,False,,
166,2,2,manager,Алло здравствуйте,True,False,False,,
250,3,1,manager,Алло дмитрий добрый день,True,False,False,,
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,True,False,True,▁максим,▁китобизнес


In [11]:
data[data.farewell==True]

,dlg_id,line_n,role,text,greeting,farewell,naming,person,company
108,0,108,manager,Всего хорошего до свидания,False,True,False,,
162,1,53,manager,Угу да вижу я эту почту хорошо тогда исправлю ...,False,True,False,,
163,1,54,manager,До свидания,False,True,False,,
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...,False,True,False,,
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...,False,True,False,,
479,5,142,manager,Ну до свидания хорошего вечера,False,True,False,,


In [12]:
data[data.naming == True]


,dlg_id,line_n,role,text,greeting,farewell,naming,person,company
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,True,ангел,
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,True,ангел,
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,True,ангелина,▁ди▁бизнес
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,True,False,True,▁максим,▁китобизнес


Вывод дополненных тестовых данных

In [13]:
data.to_csv('output.csv')

In [14]:
!head output.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
,dlg_id,line_n,role,text,greeting,farewell,naming,person,company
0,0,0,client,Алло,False,False,False,,
1,0,1,manager,Алло здравствуйте,True,False,False,,
2,0,2,client,Добрый день,False,False,False,,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,False,False,True,ангел,
4,0,4,client,Ага,False,False,False,,
5,0,5,manager,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать,False,False,False,,
6,0,6,client,Да мы работаем с компанией которая нам подливает поэтому спасибо огромное,False,False,False,,
7,0,7,client,Как как бы уже до этого момента

Создание таблицы оценок диалогов (оценка присутствия как приветствия, так и прощания)

In [15]:
decision = pd.DataFrame(data=data.dlg_id.unique(), columns=['dlg_id'])

Идентификатор диалога сделаем индексом

In [16]:
decision.set_index('dlg_id', inplace = True)

Создандим столбец оценок

In [17]:
decision['is_perfect'] = False

In [18]:
decision

,is_perfect
dlg_id,
0,False
1,False
2,False
3,False
4,False
5,False


Заполнение оценок (Истина, если есть и приветствие, и прощание)

In [19]:
for dlg_id in data.dlg_id.unique():
    dlg = data[data.dlg_id == dlg_id]
    if (dlg.greeting.sum() > 0 and dlg.farewell.sum() > 0):
        decision.at[dlg_id, 'is_perfect'] = True

In [20]:
decision

,is_perfect
dlg_id,
0,True
1,True
2,False
3,True
4,False
5,False


In [21]:
decision.to_csv('decision.csv')

In [22]:
!head decision.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
dlg_id,is_perfect
0,True
1,True
2,False
3,True
4,False
5,False
